In [ ]:
from pathlib import Path
import gzip

import bf
import kbf
import build_kmer_set
import sparse_kbf
import time
import random

from statistics import mean
import math
import pandas as pd
import pickle

You can download the data we used from Zenodo (https://doi.org/10.5281/zenodo.5760011).
You'll have to create a Data directory on the path of this notebook, and create 2 subdirectories: Genomes and Reads. Put the fasta files in the Genomes folder, fastq files in Reads folder.

In [2]:
p = Path('./Data')
g = p / 'Genomes' # directory of gzipped monkey, malaria, covid, and zebrafish genomes
r = p / 'Reads' # directory of gzipped monkey, malaria, covid, and zebrafish reads

In [3]:
def parse_fasta(fh, limit):
    count = 0
    string = []
    while limit >= count:
        seq = fh.readline()
        if len(seq) == 0:
            break
        seq = seq.decode("utf-8").rstrip()
        string.append(seq)
        count += 1
    return ["".join(string[1:])]

In [4]:
def parse_fastq(fh, limit):
    '''
    Parse reads from a FASTQ filehandle. For each read, return a name,
    nucleotide-string, quality-string triple.

    Taken from Professor Langmead.
    '''
    reads = []
    count = 0
    while limit >= count:
        first_line = fh.readline()  # ignore first line
        if len(first_line) == 0:
            break  # end of file
        seq = fh.readline()
        seq = seq.decode("utf-8").rstrip()
        fh.readline()  # ignore line starting with +
        fh.readline()  # ignore quality
        reads.append(seq)  # we only care about sequences
        count += 1
    return reads


In [5]:
def get_kmers(data, k):
    kmers = set()
    for read in data:
        for i in range(len(read) - k + 1):  # for each k-mer
            kmer = read[i:i + k]
            kmers.add(kmer)
    return kmers

In [6]:
def make_KBF(kmers, k, num_hash, size, extend_len):
    KBF = kbf.KmerBloomFilter(size, num_hash, range(num_hash), k, kmers, extend_len)
    return KBF

In [7]:
def make_KBF_Sparse(genome, k, num_hash, size, skip_len):
    best_fit_set, edge_set = build_kmer_set.best_fit_kmers(genome, k, skip_len)
    KBF_Sparse = sparse_kbf.KBFSparse(size, num_hash, 
                                     range(num_hash), k, skip_len, best_fit_set, edge_set)
    return best_fit_set, edge_set, KBF_Sparse

In [8]:
def make_all_KBFs(file, k, skip_len, num_hash, alpha, extend_len):
    fh = gzip.open(g / file, "r")
    genome = parse_fasta(fh, 10000)
    kmers = get_kmers(genome, k)
    size = round(alpha * len(kmers))
    
    BF = make_KBF(kmers, k, num_hash, size, 0)
    KBF = make_KBF(kmers, k, num_hash, size, extend_len)
    best_fit_set, edge_set, KBF_Sparse = make_KBF_Sparse(genome, k, num_hash, size, skip_len)
    
    return {
            "genome": genome, "kmers": kmers, "KBF": KBF,
            "best_fit_set": best_fit_set, "edge_set": edge_set, 
            "KBF_Sparse": KBF_Sparse, "BF": BF, "name": file
           }
    

In [126]:
num_hash = 6
k = 20
alpha = 5
skip_len = 3
extend_len = 2

In [10]:
plas_Bloom = make_all_KBFs('plasmodium.fa.gz', k, skip_len, num_hash, alpha, extend_len)

In [11]:
covid_Bloom = make_all_KBFs('SARS-CoV-2.fa.gz', k, skip_len, num_hash, alpha, extend_len)

In [12]:
zfish_Bloom = make_all_KBFs('Zebrafish.fa.gz', k, skip_len, num_hash, alpha, extend_len)

In [13]:
monkey_Bloom = make_all_KBFs('monkey.fa.gz', k, skip_len, num_hash, alpha, extend_len)

In [15]:
KMERS = {}
for file in r.iterdir():
    fh = gzip.open(file, "r")
    reads = parse_fastq(fh, 10000)
    kmers = get_kmers(reads, k)
    KMERS[file] = kmers


All_Classifications = {}
for Bloom in [monkey_Bloom, plas_Bloom, covid_Bloom, zfish_Bloom]:
    All_Classifications[Bloom["name"]] = {}
    
    for KBF_Implementation in ["BF", "KBF", "KBF_Sparse"]:
        All_Classifications[Bloom["name"]][KBF_Implementation] = {}
        
        classification = {}
        for file in r.iterdir():
            classification[file] = [] 
            
        FPRs = []
        Runtimes = []
        
        for i in range(5): 
            for file in r.iterdir():
                print(f"{i + 1} {file}")
                kmers = KMERS[file]

                tp = 0
                tn = 0
                fp = 0
                fn = 0
                pos = 0

                start = time.time()
                for x in random.sample(kmers, 100000):
                    truth = x in Bloom["kmers"]
                    if x in Bloom[KBF_Implementation]:
                        pos += 1
                        if truth:
                            tp += 1
                        else:
                            fp += 1
                    else:
                        if truth:
                            fn += 1
                        else:
                            tn += 1
                end = time.time()

                classification[file].append(pos)
                FPRs.append(fp / (fp + tn))
                Runtimes.append(end - start)
                
        All_Classifications[Bloom["name"]][KBF_Implementation]["classification"] = classification
        All_Classifications[Bloom["name"]][KBF_Implementation]["FPRs"] = FPRs
        All_Classifications[Bloom["name"]][KBF_Implementation]["Runtimes"] = Runtimes
        
        print("Bloom: " + Bloom["name"] + ", KBF: " + KBF_Implementation)
        print(All_Classifications[Bloom["name"]][KBF_Implementation])

1 Data/Reads/sars-cov-2.fastq.gz


/var/folders/bv/z4chszkx0gx32hlqmjdzsc5c0000gn/T/ipykernel_80867/3122238093.py:35: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  for x in random.sample(kmers, 100000):


1 Data/Reads/plasmodium.fastq.gz
1 Data/Reads/zebrafish.fastq.gz
1 Data/Reads/monkey.fastq.gz
2 Data/Reads/sars-cov-2.fastq.gz
2 Data/Reads/plasmodium.fastq.gz
2 Data/Reads/zebrafish.fastq.gz
2 Data/Reads/monkey.fastq.gz
3 Data/Reads/sars-cov-2.fastq.gz
3 Data/Reads/plasmodium.fastq.gz
3 Data/Reads/zebrafish.fastq.gz
3 Data/Reads/monkey.fastq.gz
4 Data/Reads/sars-cov-2.fastq.gz
4 Data/Reads/plasmodium.fastq.gz
4 Data/Reads/zebrafish.fastq.gz
4 Data/Reads/monkey.fastq.gz
5 Data/Reads/sars-cov-2.fastq.gz
5 Data/Reads/plasmodium.fastq.gz
5 Data/Reads/zebrafish.fastq.gz
5 Data/Reads/monkey.fastq.gz
Bloom: monkey.fa.gz, KBF: BF
{'classification': {PosixPath('Data/Reads/sars-cov-2.fastq.gz'): [12210, 12103, 12173, 12172, 12130], PosixPath('Data/Reads/plasmodium.fastq.gz'): [12200, 12204, 12311, 12208, 12284], PosixPath('Data/Reads/zebrafish.fastq.gz'): [11544, 11508, 11624, 11558, 11763], PosixPath('Data/Reads/monkey.fastq.gz'): [12441, 12584, 12435, 12250, 12502]}, 'FPRs': [0.11840612164971

In [49]:
filehandler = open(p / 'Results/All_Classifications.pkl', 'wb') 
pickle.dump(All_Classifications, filehandler)
filehandler.close()

In [131]:
Results = pickle.load(open(p / 'Results/All_Classifications.pkl', 'rb'))

In [132]:
Genomes = ["M. mulatta", "P. falciparum", "SARS-CoV-2", "D. rerio"]
BF_FPRs = []
KBF_FPRs = []
Sparse_FPRs = []
BF_Runtimes = []
KBF_Runtimes = []
Sparse_Runtimes = []


Kmer_lens = []
Bestfit_lens = []
Sizes = []

for Bloom in [monkey_Bloom, plas_Bloom, covid_Bloom, zfish_Bloom]:
    Kmer_lens.append(len(Bloom["kmers"]))
    Bestfit_lens.append(len(Bloom["best_fit_set"]))
    Sizes.append(round(len(Bloom["kmers"]) * alpha))

In [134]:
for x in r.iterdir():
    print(f"Classifying species of {x}...")
    for kbf_type in ["BF", "KBF", "KBF_Sparse"]:
        print(f"Via {kbf_type} implementation")
        max_hit = -math.inf
        max_word = ""
        for genome_type in Results:
            hits = mean(Results[genome_type][kbf_type]["classification"][x])
            if hits > max_hit:
                max_hit = hits
                max_word = genome_type.split(".")[0].lower()
            # print(f"genome: {genome_type}, # of hits: {hits}")
        if max_word == str(x).split("/")[-1].split(".")[0].lower():
            print(f"great! we classified correctly: {max_word}")
        else:
            print(f"WARNING: WRONG CLASSIFICATION: {max_word}")
    print('\n')
            
for key in ["monkey", "plasmodium", "SARS-CoV-2", "Zebrafish"]:
    genome_type = key + ".fa.gz"
    for kbf_type in Results[genome_type]:
        core_result = Results[genome_type][kbf_type]
        
        if kbf_type == "BF":
            BF_FPRs.append(mean(core_result["FPRs"]))
            BF_Runtimes.append(mean(core_result["Runtimes"]))
        elif kbf_type == "KBF":
            KBF_FPRs.append(mean(core_result["FPRs"]))
            KBF_Runtimes.append(mean(core_result["Runtimes"]))
        elif kbf_type == "KBF_Sparse":
            Sparse_FPRs.append(mean(core_result["FPRs"]))
            Sparse_Runtimes.append(mean(core_result["Runtimes"]))

Classifying species of Data/Reads/sars-cov-2.fastq.gz...
Via BF implementation
great! we classified correctly: sars-cov-2
Via KBF implementation
great! we classified correctly: sars-cov-2
Via KBF_Sparse implementation
great! we classified correctly: sars-cov-2


Classifying species of Data/Reads/plasmodium.fastq.gz...
Via BF implementation
great! we classified correctly: plasmodium
Via KBF implementation
great! we classified correctly: plasmodium
Via KBF_Sparse implementation
great! we classified correctly: plasmodium


Classifying species of Data/Reads/zebrafish.fastq.gz...
Via BF implementation
great! we classified correctly: zebrafish
Via KBF implementation
great! we classified correctly: zebrafish
Via KBF_Sparse implementation
great! we classified correctly: zebrafish


Classifying species of Data/Reads/monkey.fastq.gz...
Via BF implementation
great! we classified correctly: monkey
Via KBF implementation
great! we classified correctly: monkey
Via KBF_Sparse implementation
great! we

In [135]:
pd.options.display.float_format = '{:,.3f}'.format
FPR_Dataset = list(zip(Genomes,BF_FPRs,KBF_FPRs,Sparse_FPRs))
df = pd.DataFrame(data = FPR_Dataset, columns=['Genome', 'KBF', '2-KBF', 'Sparse KBF'])
df

,Genome,KBF,2-KBF,Sparse KBF
0,M. mulatta,0.116,0.100,0.000
1,P. falciparum,0.116,0.101,0.000
2,SARS-CoV-2,0.116,0.100,0.000
3,D. rerio,0.117,0.101,0.000


In [136]:
Runtime_Dataset = list(zip(Genomes,BF_Runtimes,KBF_Runtimes,Sparse_Runtimes))
df = pd.DataFrame(data = Runtime_Dataset, columns=['Genome', 'KBF', '2-KBF', 'Sparse KBF'])
df

,Genome,KBF,2-KBF,Sparse KBF
0,M. mulatta,0.410,0.903,7.987
1,P. falciparum,0.392,0.906,7.909
2,SARS-CoV-2,0.367,0.937,7.089
3,D. rerio,0.412,0.919,8.048


In [137]:
Size_Dataset = list(zip(Genomes,Kmer_lens,Bestfit_lens,Sizes))
df = pd.DataFrame(data = Size_Dataset, 
                  columns=['Genome', '# of k-mers', '# of k-mers (sparse)', 'KBF Size'])
for key in df:
    if key != 'Genome':
        df[key] = df[key].apply('{:,}'.format)
df

,Genome,# of k-mers,# of k-mers (sparse),KBF Size
0,M. mulatta,"649,964","168,543","3,249,820"
1,P. falciparum,"571,486","149,881","2,857,430"
2,SARS-CoV-2,"29,871","7,469","149,355"
3,D. rerio,"629,173","163,419","3,145,865"


We want to see if we can use a smaller sparse KBF to achieve similar FPR as a bigger two-sided KBF.
Change alpha to 2.

In [138]:
alpha = 2 # decreased from 5 to 2, size of KBF will be reduced by a factor of 2.5

In [ ]:
plas_Bloom = make_all_KBFs('plasmodium.fa.gz', k, skip_len, num_hash, alpha, extend_len)
covid_Bloom = make_all_KBFs('SARS-CoV-2.fa.gz', k, skip_len, num_hash, alpha, extend_len)
zfish_Bloom = make_all_KBFs('Zebrafish.fa.gz', k, skip_len, num_hash, alpha, extend_len)
monkey_Bloom = make_all_KBFs('monkey.fa.gz', k, skip_len, num_hash, alpha, extend_len)

KMERS = {}
for file in r.iterdir():
    fh = gzip.open(file, "r")
    reads = parse_fastq(fh, 10000)
    kmers = get_kmers(reads, k)
    KMERS[file] = kmers


All_Classifications = {}
for Bloom in [monkey_Bloom, plas_Bloom, covid_Bloom, zfish_Bloom]:
    All_Classifications[Bloom["name"]] = {}
    
    for KBF_Implementation in ["KBF_Sparse"]:
        All_Classifications[Bloom["name"]][KBF_Implementation] = {}
        
        classification = {}
        for file in r.iterdir():
            classification[file] = [] 
            
        FPRs = []
        Runtimes = []
        
        for i in range(5): 
            for file in r.iterdir():
                print(f"{i + 1} {file}")
                kmers = KMERS[file]

                tp = 0
                tn = 0
                fp = 0
                fn = 0
                pos = 0

                start = time.time()
                for x in random.sample(kmers, 100000):
                    truth = x in Bloom["kmers"]
                    if x in Bloom[KBF_Implementation]:
                        pos += 1
                        if truth:
                            tp += 1
                        else:
                            fp += 1
                    else:
                        if truth:
                            fn += 1
                        else:
                            tn += 1
                end = time.time()

                classification[file].append(pos)
                FPRs.append(fp / (fp + tn))
                Runtimes.append(end - start)
                
        All_Classifications[Bloom["name"]][KBF_Implementation]["classification"] = classification
        All_Classifications[Bloom["name"]][KBF_Implementation]["FPRs"] = FPRs
        All_Classifications[Bloom["name"]][KBF_Implementation]["Runtimes"] = Runtimes
        
        print("Bloom: " + Bloom["name"] + ", KBF: " + KBF_Implementation)
        print(All_Classifications[Bloom["name"]][KBF_Implementation])

In [97]:
filehandler = open(p / 'Results/Sparse_Classifications_Smaller.pkl', 'wb') 
pickle.dump(All_Classifications, filehandler)
filehandler.close()

In [ ]:
Results = pickle.load(open(p / 'Results/Sparse_Classifications_Smaller.pkl', 'rb'))

In [140]:
Genomes = ["M. mulatta", "P. falciparum", "SARS-CoV-2", "D. rerio"]
Sparse_FPRs = []
Sizes = []

for Bloom in [monkey_Bloom, plas_Bloom, covid_Bloom, zfish_Bloom]:
    Sizes.append(round(len(Bloom["kmers"]) * alpha))

In [141]:
for key in ["monkey", "plasmodium", "SARS-CoV-2", "Zebrafish"]:
    genome_type = key + ".fa.gz"
    core_result = Results[genome_type]["KBF_Sparse"]
    Sparse_FPRs.append(mean(core_result["FPRs"]))

In [142]:
FPR_Dataset = list(zip(Genomes,Sparse_FPRs,Sizes))
df = pd.DataFrame(data = FPR_Dataset, columns=['Genome', 'FPR', 'Size'])
df['Size'] = df['Size'].apply('{:,}'.format)
df

,Genome,FPR,Size
0,M. mulatta,0.129,"1,299,928"
1,P. falciparum,0.140,"1,142,972"
2,SARS-CoV-2,0.101,"59,742"
3,D. rerio,0.131,"1,258,346"
